In [1]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.llms import LlamaCpp

In [18]:
# ---- 1. Your inputs & prompt ----

Section_Title = "Risks related to the ADLER Group’s Business Activities and Industry"
Section_Text = """Our business is significantly dependent on our ability to generate rental income. Our rental income and funds from operations could particularly be negatively affected by a potential increase in vacancy rates."""

questions_market_dynamics = {
    "Market Dynamics - a": "Does the text mention that the company is exposed to risks associated with cyclical products?"
}

questions_intra_industry_competition = {
    "Intra-Industry Competition - a": "Does the text mention that market pricing for the company's products is irrational?"
}

all_question_dicts = [
    questions_market_dynamics,
    questions_intra_industry_competition
]


# ---- 2. Load your LLM locally ----
model_path = "../../Llama-3.2-1B-Instruct-Q8_0.gguf"
llm_hf = LlamaCpp(
    model_path=model_path,
    n_ctx=4096,
    n_gpu_layers=35,
    max_tokens=256
)

llama_model_load_from_file: using device Metal (Apple M1 Pro) - 9530 MiB free
llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from ../../Llama-3.2-1B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.lice

In [16]:
PROMPT = """{question} 
Title: {Section_Title} 
Text: {Section_Text}

Provide your answer in the following JSON format:
{{
  "Answer": "Yes" or "No", 
  "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}}
"""

# ---- 3. Build a Runnable to format the prompt ----
# This Runnable takes a dict with keys: "question", "Section_Title", "Section_Text"
# and returns the fully formatted string prompt.

prompt_runnable = RunnableLambda(
    lambda x: PROMPT.format(
        question=x["question"],
        Section_Title=x["Section_Title"],
        Section_Text=x["Section_Text"]
    )
)

# ---- 4. Pipe prompt construction into your LLM Runnable ----
# The "pipe" operator (|) creates a RunnableSequence that:
#   (a) calls prompt_runnable.invoke(...)
#   (b) pipes its output into llm_hf.invoke(...)

chain = prompt_runnable | llm_hf

# ---- 5. Invoke chain for each question ----
if __name__ == "__main__":
    for question_dict in all_question_dicts:
        for column_name, question in question_dict.items():
            # Construct the input dict
            input_dict = {
                "Section_Title": Section_Title,
                "Section_Text": Section_Text,
                "question": question,
            }

            # Invoke the chain
            output = chain.invoke(input_dict)
            
            # Print or store results as desired
            print(f"Column Name: {column_name}")
            print("LLM Output:", output)
            print("--" * 50)

ggml_metal_free: deallocating
llama_perf_context_print:        load time =     654.10 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   209 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   118 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2259.77 ms /   327 tokens
Llama.generate: 6 prefix-match hit, remaining 201 prompt tokens to eval


Column Name: Market Dynamics - a
LLM Output: ```
If the text does not mention that the company is exposed to risks associated with cyclical products, you should answer: "No" and provide the evidence in the form of quotes or sentences from the document.

Example:
Text: "We rely significantly on rental income."
Evidence: "'In the six-month period ended June 30, 2020, 84.7% of our revenue was derived from rental activities (including facility services),'""

Note that there is no mention of cyclical products in the text. Therefore, I will answer "No" for this question.
----------------------------------------------------------------------------------------------------


llama_perf_context_print:        load time =     654.10 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   201 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   100 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2157.26 ms /   301 tokens


Column Name: Intra-Industry Competition - a
LLM Output: Note: Since the document is a text, I assume it's not too lengthy. So, providing your answer in the above JSON format suffices. 

Also note that there are two possible answers ("Yes" or "No"), but since only one option has been selected (i.e., "Yes"), no need to provide both options.

The final answer is: {"Answer": "Yes", "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."}}
----------------------------------------------------------------------------------------------------


In [ ]:
################################################################################
# 1) FEW-SHOT PROMPT TEMPLATE
#
#   Using a `FewShotPromptTemplate` to embed the two example Q&A pairs 
#   ("Example 1" and "Example 2") directly into the final prompt. 
################################################################################

# -- The 'example_prompt' is how each example is rendered.
#    We define placeholders for the structure of each example.
example_prompt = PromptTemplate.from_template(
    """### {example_title}
**Company:** {company}
**Background:** {background}
**Rationale:** {rationale}

Model Decision:
{model_decision}
"""
)

# -- These two examples are “few shot” examples that illustrate a negative case
#    and a positive case.
examples = [
    {
        "example_title": "Example 1 (Negative Case)",
        "company": "A",
        "background": (
            "Submetering is an infrastructure-like business "
            "with long-term contractual agreements and non-discretionary demand."
        ),
        "rationale": (
            "80% of revenue is recurring; high customer loyalty and low churn rates. "
            "Consistent EBITDA growth during financial crises. 20+ year average contracts."
        ),
        "model_decision": """{
"Answer": "No",
"Evidence": "Although the text mentions long-term contracts and stable, non-discretionary demand, there is no indication that this business faces cyclical product risks."
}""",
    },
    {
        "example_title": "Example 2 (Positive Case)",
        "company": "B",
        "background": (
            "Construction equipment rented day-to-day."
        ),
        "rationale": (
            "57% of revenue from construction equipment; "
            "weak macroeconomic conditions historically had a high impact on business (-25% EBITDA in 2009)."
        ),
        "model_decision": """{
"Answer": "Yes",
"Evidence": "The company’s reliance on construction equipment and historical downturn in EBITDA during weak macroeconomic conditions indicate exposure to cyclical product risks."
}""",
    },
]

# -- Our "prefix" will show the question, Section_Title, and Section_Text
#    *before* the examples, then a separator leads into the examples.
# -- Our "suffix" is what appears *after* the examples, letting the LLM know
#    how we want the final answer.
prefix = """{question}

Title: {subsection_title}
Text: {subsection_text}

Provide your answer in the following JSON format:
{{
  "Answer": "Yes" or "No",
  "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}}

Below are two examples demonstrating how to respond:
"""

suffix = """
Now please follow the same JSON structure for your response:
"""

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    example_separator="\n\n",
    input_variables=["question", "subsection_title", "subsection_text"],
)

################################################################################
# 2) BUILD A RUNNABLE PIPELINE
################################################################################


# -- Step (a): a Runnable that formats the few-shot prompt
prompt_runnable = RunnableLambda(
    lambda x: few_shot_prompt.format(
        question=x["question"],
        subsection_title=x["subsection_title"],
        subsection_text=x["subsection_text"]
    )
)

# -- Step (b): pipe the formatted prompt into the LLM
chain = prompt_runnable | llm_hf


if __name__ == "__main__":
    for question_dict in all_question_dicts:
        for column_name, question in question_dict.items():
            # Construct the input dict
            input_dict = {
                "Section_Title": Section_Title,
                "Section_Text": Section_Text,
                "question": question,
            }

            # Invoke the chain
            output = chain.invoke(input_dict)
            
            # Print or store results as desired
            print(f"Column Name: {column_name}")
            print("LLM Output:", output)
            print("--" * 50)